# Data cleansing project: Shark attacks



Inicio:

+ Exploración inicial. Describe, columns, value_counts() etc.
+ Filas duplicadas.

Posibles objetivos: 

+ Hora del día o condiciones lumínicas a la que ocurren más los ataques (hora, sitio, tipos)
+ Agresividad de los distintos tiburones
+ Actividades peligrosas
+ Países con más ataques
+ 

Pasos:

+ Valores nulos.
+ Datos inconsistentes (dividir información compuesta).
+ Data types. Memory usage
+ Valores atípicos (outliers).
+ 

In [1]:
# Librerias y settings generales:

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import zipfile
import chardet
import regex as re

import warnings
warnings.filterwarnings('ignore')

# gráficos
import pylab as plt    # import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Descomprimimos y guardamos el archivo en un csv. 
# Da un error al leerlo desde read_csv si no se maneja el encoding_error de algunas entradas. 
# Poniendo replace las localizamos.

zf = zipfile.ZipFile('./archive.zip')
shark = pd.read_csv(zf.open('attacks.csv'), encoding = 'latin-1')

# Guardamos el original antes de seguir:

shark_ori = shark.copy()

shark.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [3]:
# Quitamos duplicados si los hay:

shark.drop_duplicates(inplace = True)

# Si hay filas que tienen TODAS las columnas nulas, las eliminamos:

shark.dropna(axis = 0, how="all",inplace = True)

shark.shape, shark_ori.shape # Nos hemos quitado la mayoría

((6311, 24), (25723, 24))

In [4]:
# Limpiamos los nombres de las columnas, quitando espacios innecesarios

shark.columns = [i.lower().replace(' ','_') for i in shark.columns]
colnames = list(shark.columns)
colnames[9] = 'sex'
colnames[12] = 'fatal'
colnames[14] = 'species'

shark.columns = colnames
shark.head()

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [5]:
# Miramos a ver si hay columnas de valor constante que no aporten información sobre la influencia de factores

constant_col = []

for i in shark:
    if len(shark[i].unique()) == 1:
        constant_col.append(i)

constant_col # Parece que estrictamente constantes no hay

[]

In [6]:
# Empezamos una limpieza básica de las columnas

# COLUMNA 'sex'

print(shark.sex.value_counts())

# Cambiamos el sexo 'N' por 'M', es probable que sea un error de teclado ya que las teclas de ambas letras
# son contiguas. El sexo 'M ' es el mismo que 'M':

shark.sex[shark.sex == 'N'] = 'M' 
shark.sex[shark.sex == 'M '] = 'M' 

# Quedan dos valores raros, 'lli' y '.', que aparecen solo una vez. 
# El valor '.' corresponde a restos de 3 personas, por lo que dejamos la fila pendiente de
# quitar por si no nos sirve para el objetivo y asignamos el valor 'unknown'.
# El valor 'lli' corresponde a al ataque a Brian Kang. Asumimos sexo masculino, y sustituimos por 'M'

shark.sex[shark.sex == '.'] = 'unknown' 
shark.sex[shark.sex == 'lli'] = 'M' 

# Ahora quedan solo dos valores posibles. Observamos que el número de ataques masculinos es mucho mayor que el 
# número de ataques femeninos, lo que podría tener muchas causas, entre ellas el número de personas 
# de cada sexo practicando actividades de alta o baja peligrosidad

print('\n')
print(shark.sex.value_counts())


M      5094
F       637
M         2
N         2
lli       1
.         1
Name: sex, dtype: int64


M          5099
F           637
unknown       1
Name: sex, dtype: int64


In [7]:
# COLUMNA 'fatal'

print(shark.fatal.value_counts())

shark.fatal[shark.fatal == ' N'] = 'N'
shark.fatal[shark.fatal == 'N '] = 'N'
shark.fatal[shark.fatal == 'y'] = 'Y'
shark.fatal[shark.fatal == 'M'] = 'N' # Asumimos error de introducción de datos, como antes
shark.fatal[shark.fatal == '2017'] = 'UNKNOWN' # Descripción no clara, dato desconocido

# Ahora pasamos 'UNKNOWN' a minúsculas y cambiamos Y/N por un booleano True/False:

shark.fatal[shark.fatal == 'UNKNOWN'] = 'unknown'
shark.fatal[shark.fatal == 'Y'] = True
shark.fatal[shark.fatal == 'N'] = False

shark.fatal.value_counts()

N          4293
Y          1388
UNKNOWN      71
 N            7
M             1
2017          1
N             1
y             1
Name: fatal, dtype: int64


False      4302
True       1389
unknown      72
Name: fatal, dtype: int64

In [8]:
# COLUMNA 'case_number'

print(shark.case_number.value_counts()) 

0               8
1920.00.00.b    2
1966.12.26      2
2014.08.02      2
1990.05.10      2
               ..
1999.09.05      1
1999.09.10      1
1999.09.16      1
1999.09.18      1
xx              1
Name: case_number, Length: 6287, dtype: int64


In [9]:
# Las filas que tienen case_number = 0 son muy sospechosas. 
# Cuando las examinamos, vemos que son filas nulas en todas las columnas salvo en 'original_order'. 
# Estas filas no aportan información, se van fuera

#r'\d{4}-\d{2}-\d{2}'

shark.drop(index = shark[shark.case_number == '0'].index, inplace = True)
shark.shape, shark_ori.shape
print(shark.case_number.value_counts()) 

1920.00.00.b      2
1915.07.06.a.R    2
2009.12.18        2
2014.08.02        2
1962.06.11.b      2
                 ..
1999.09.05        1
1999.09.10        1
1999.09.16        1
1999.09.18        1
xx                1
Name: case_number, Length: 6286, dtype: int64


In [10]:
# Con respecto al valor 'xx', nos fijamos en el resto de esa fila:

display(shark[shark_ori['Case Number']=='xx'])

# Vemos que es nula, así que fuera.

shark.drop(index = shark[shark_ori['Case Number']=='xx'].index, inplace = True)

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:


# OJOOOOOOOOOOOOOOOOOOOOOOOOOOOOO QUITAR


# La columna 'case_number' contiene la fecha del registro en un formato mucho más fácil de procesar que
# la columna 'date'. 

# Ahora definimos una función para procesar la información de la columna 'case_number':

'''def clean_date(x):
    
    try:
        num = re.findall('\d+', x) # Buscamos una lista con los números que haya
        try:
            year = int(num[0])
            month = int(num[1])
            day = int(num[2]) 
        except:
            try:
                year = int(num[0])
                month = int(num[1])
                day = 'unknown'
            except:
                try:
                    year = int(num[0])
                    month = 'unknown'
                    day = 'unknown'
                except:
                        year = -1
                        month = -1
                        day = -1                    
                    
        res = [day,month,year]
            
    except: # Por si patata
        res = [-1,-1,-1]
        
    return res'''

# Y reescribimos cada columna con ella:

'''sharkk.day = [clean_date(e)[0] for e in sharkk.case_number]
sharkk.month = [clean_date(e)[1] for e in sharkk.case_number]
sharkk.year = [clean_date(e)[2] for e in sharkk.case_number]'''


'sharkk.day = [clean_date(e)[0] for e in sharkk.case_number]\nsharkk.month = [clean_date(e)[1] for e in sharkk.case_number]\nsharkk.year = [clean_date(e)[2] for e in sharkk.case_number]'

In [12]:
# Revisamos las que han salido rana, es decir, las etiquetadas con '-1':


In [13]:
# COLUMNA 'original_order'

print(shark.original_order.value_counts()) 

# Parece que esta columna es el orden original de registros, y que solo hay uno repetido. 
# Ordenamos la tabla según esta columna y la dejamos estar

shark.sort_values('original_order', ascending = True, inplace = True)
shark.reset_index (inplace = True, drop = True)

569.0     2
6303.0    1
2106.0    1
2097.0    1
2098.0    1
         ..
4205.0    1
4206.0    1
4207.0    1
4208.0    1
2.0       1
Name: original_order, Length: 6301, dtype: int64


In [14]:
# Limpiamos un poco más las filas. En aquellas en las que haya un porcentaje de columnas con más del 50% de
# nulos van fuera (solo queda una):

na_rows = shark.isna().mean(axis = 1)*100
na_rows[na_rows>50]
shark.drop(index = na_rows[na_rows>50].index, inplace = True)

shark.shape, shark_ori.shape

((6302, 24), (25723, 24))

In [15]:
# Vamos a por los valores nulos.

shark.isna().sum()

case_number                  1
date                         0
year                         2
type                         4
country                     50
area                       455
location                   540
activity                   544
name                       210
sex                        565
age                       2831
injury                      28
fatal                      539
time                      3354
species                   2838
investigator_or_source      17
pdf                          0
href_formula                 1
href                         0
case_number.1                0
case_number.2                0
original_order               0
unnamed:_22               6301
unnamed:_23               6300
dtype: int64

In [16]:
# En este punto, sería interesante ocuparnos sólo de los valores nulos que realmente vayan a influir. 
# Fijamos un objetivo:

# AVERIGUAR LA RELACIÓN ENTRE LA SUPERVIVENCIA AL ATAQUE DE UN TIBURÓN Y LA EDAD DE LA PERSONA ATACADA

# Las columnas que más nos interesan en este sentido:

# La edad
# La supervivencia
# El tipo de herida (buscaremos casos de heridas graves, no simples laceraciones)

# Eliminamos las filas que no tengan dato en alguna de las dos columnas más importantes, edad y supervivencia
# (previamente se han checkeado los casos en los que la edad falta como dato, y no se encuentra en otras columnas):

shark.drop(index = shark[(shark.fatal.isna())|(shark.age.isna())].index, inplace = True)

shark.shape, shark_ori.shape

((3244, 24), (25723, 24))

In [17]:
# Echemos un ojo a las columnas clave para limpiar su información

# Columna EDAD:

shark.age.unique()

array(['15', '16', '13 or 14', '50', '6', '25', '36', '27', '19', '21',
       '20', '14', 'young', '22', '17', '52', '12', '29', '7 or 8', '2½',
       '18', '28', '13', '35', '40', '39', '30', '3', '11', '23', '60',
       '37', '33', '26', '24', '9', 'Teen', '10', '31 or 33', '55', '64',
       '7', '43', '10 or 12', '41', '38', '47', '8', '54', '?    &   14',
       'A.M.', '70', '78', '  ', '42', '50s', '49', '45', '36 & 23', '32',
       '34', '31', '9 or 10', 'Both 11', '53', '48', 'F', '8 or 10',
       '17 & 16', '46', '"young"', '56', '81', ' 43', 'MAKE LINE GREEN',
       '61', '62', '57', '59', '33 & 26', '13 or 18', '"middle-age"', 'X',
       '17 & 35', '50 & 30', '5', '30 & 32', '44', '58', '25 or 28',
       '33 & 37', '(adult)', '1', '23 & 26', '25 to 35', '9 months',
       '9 & 12', '65', 'adult', '18 to 22', '66', '>50', '51', '20s',
       '20 ', '21 or 26', '45 ', '74 ', ' ', '75', 'Elderly', '87', '30s',
       '32 & 30', '69', "60's", '71', '20?', ' 28', '7     

In [18]:
# Vemos que hay varios casos que aportan información imprecisa, como 'teens', 'eldery' o '(adult)'.
# También hay casos que describen dos ataques a la vez, y casos en los que la edad está expresada en meses.
# Identificaremos y gestionaremos todos los casos con una función basada en regex, que haga lo siguiente:

# Devolver el entero 0 si la información es ambigua (no numérica)
# Devolver el entero 0 si nos hablan de 'middle age'. 'mid-30s' también es muy ambiguo
# Devolver el entero 0 si la información está en meses (no incluiremos niños tan pequeños en el estudio)
# Devolver un entero redondeado con la media de las edades si la información es del tipo '13 or 14'
# Devolver el entero 0 si la información es sobre dos personas distintas, ya que son pocos casos y dificultarán 
# obtener conclusiones sobre ellos a partir del resto de columnas


def clean_age(x):
    if 'month' in x or 'mid' in x:
        res = 0
    elif 'or' in x:
        ages = re.findall('\d+', x)
        res0 = [int(e) for e in ages]
        res = int(np.mean(res0))
    else:
        ages = re.findall('\d+', x)
        if len(ages) == 1:
            res = int(ages[0])
        else:
            res = 0
            
    return res

shark.age = [clean_age(x) for x in shark.age]

shark.age.unique()

array([15, 16, 13, 50,  6, 25, 36, 27, 19, 21, 20, 14,  0, 22, 17, 52, 12,
       29,  7,  2, 18, 28, 35, 40, 39, 30,  3, 11, 23, 60, 37, 33, 26, 24,
        9, 10, 32, 55, 64, 43, 41, 38, 47,  8, 54, 70, 78, 42, 49, 45, 34,
       31, 53, 48, 46, 56, 81, 61, 62, 57, 59,  5, 44, 58,  1, 65, 66, 51,
       74, 75, 87, 69, 71, 63, 73, 84, 77, 68, 86, 72, 82])

In [19]:
# Eliminamos todas las filas que tengan edad 0, que son las que hemos etiquetado para descartar:

shark.drop(index = shark[shark.age == 0].index, inplace = True)

shark.shape, shark_ori.shape

((3195, 24), (25723, 24))

In [20]:
# Ya tenemos las dos columnas principales, age y fatal, limpias de valores nulos y procesadas para obtener
# información. Echamos ahora un vistazo al tipo de herida:

shark.injury.unique()[:20]

array(['FATAL. "Shark bit him in half, carrying away the lower extremities" ',
       'FATAL', 'FATAL, left leg bitten with severe blood loss',
       'FATAL, hip bitten  PROVOKED INCIDENT',
       'Right leg lacerated & surgically amputated', 'Arm severed',
       'Right hand severed', 'Bumped by sharks',
       'No injury, bumped by shark which took speared fish',
       'Ankle punctured & lacerated, hands abraded PROVOKED INCIDENT',
       'Severe abrasion to forearm from captive shark PROVOKED INICIDENT',
       'Foot severed',
       'Hand and foot severely bitten, surgically amputated',
       'Right leg severed at knee.  In 1796 he became Lord Mayor of London. In 1778 he commissioned  American artist, John Singleton Copley, to paint the incident: Watson and the Shark',
       'Left arm severed 2.5" from elbow, groin, abdomen, right forearm & hand lacerated, flesh removed from thigh, baring femur. Both arms surgically amputated: left arm above elbow, right arm above wrist. Surviv

In [21]:
# Hay muchísima variedad. En términos de supervivencia según la edad, nos interesan aquellos registros en los que
# el daño haya sido considerable, no un simple rasguño. Echando un ojo a las descripciones que contienen 
# 'no' + 'injury','minor' o 'slight', observamos que corresponden a ataques sin daño personal o con daños menores:

c = 0
a = []
b = []

for i in shark.injury.unique():
    try:
        if 'no' in i.lower() and 'injury' in i.lower() or 'minor' in i.lower() or 'slight' in i.lower():
            print(i)
            b.append(shark[shark.injury == i].index) # Nos guardamos los índices en 'b' para quitar filas luego
        else: 
            pass
        
    except: # Para localizar objetos no-string y posibles valores nulos
        a.append(i)

print(a)


No injury, bumped by shark which took speared fish
Minor injuries
No injury, shark tore hole in the side of the boat
No injury to occupants, shark bit keel
Minor injury
No injury, hooked shark rammed tripod PROVOKED INCIDENT
Torso bitten with pneumothorax, slight lacerations on left hand
Shark rammed boat; no injury to occupants
No injury
Minor injury to shoulder & back PROVOKED INCIDENT
Minor injury, ankle scratched by shark's teeth
No injury to occupant, shark grabbed rudder and dragged the dinghy stern-first
No injury; shark rammed boat, catapulting Leverenz in the sea & damaging boat
No injury, board scraped by shark
No injury, ski bumped
No injury,     Shark landed on top of him, knocked him back down 3 times
No injury to occupants, shark holed canoet
Minor injury to hand and groin from shark "caught on his fishing spear" PROVOKED INCIDENT
No injury, shark bit oar
Minor laceration to chest when he grabbed the shark by its tail PROVOKED INCIDENT
Minor bruises & abrasions on leg
Div

In [22]:
# Con este bucle, confirmamos que todas estas descripciones son de daños personales menores o nulos.
# También nos sobran valores nulos, cuya presencia se ha detectado en el bucle.
# Reescribimos estos últimos con 'unknown', y quitamos las filas de los de daños menores:

shark.injury = shark.injury.fillna('unknown')

for i in b:
    shark.drop(index = i, inplace = True)

shark.shape, shark_ori.shape

# De los ataques que quedan no todos serán graves, pero sí la mayoría, y nos servirán para 
# obtener alguna conclusión estadística.

((2797, 24), (25723, 24))

In [23]:
# Pasamos ahora a la columna del año del ataque, para descartar los casos anteriores a 1900 y centrar
# el estudio en registros más actuales. La columna de 'year' tiene valores absurdos. Si calculamos el mínimo
# obtenemos 0:

shark.year.min()

# Por eso, vamos a limpiar y procesar ahora la columna 'date'.

0.0

In [24]:
# PROCESADO DE FECHAS

# Columnas 'date' y 'year':

# Vamos a rellenar las columnas libres que tenemos, 'unnamed:_22' y 'unnamed:_23', 
# con el día y el mes, e iremos sobreescribiendo también el año en la columna 'year' en que hay algunas
# entradas falsas que tienen valor 0 y demás

# Primero renombramos estas dos columnas libres:

col_names = list(shark.columns)
col_names[-1] = 'month'
col_names[-2] = 'day'
shark.columns = [e for e in col_names]

# Ahora, como sólo nos vamos a quedar con las fechas superiores a 1900, hacemos un primer filtro. 
# Buscaremos números de cuatro dígitos superiores o iguales a 1900 en 'date', 'case_number', 'case_number.1',
# 'case_number.2' y 'year'. Si no lo encontramos en ninguna de esas columnas, marcaremos 'year' con -1 para 
# eliminar el registro.

def locate_highyear(x):
    num = re.findall('\d+', x) # Hacemos una lista con los números que haya en x
    res = 0
    for i in num:
        try:
            if int(i)>=1900:
                res+=1 # Si alguno pasa de 1900, hacemos crecer el contador
            else:
                pass
        except:
            pass
    return res

for i in shark.index:
    if sum([locate_highyear(shark.date[i]),locate_highyear(shark.case_number[i]),
           locate_highyear(shark['case_number.1'][i]),locate_highyear(shark['case_number.2'][i])]) == 0:
        shark.drop(index = i, inplace = True)
    else:
        pass
    
shark.shape, shark_ori.shape

((2722, 24), (25723, 24))

In [25]:
for i in shark.date.unique():
    print(i)

Early 1930s
1941-1942
Before 1957
1960s
Before 1962
No date, Before  1975
Before 2009
Before Oct-2009
14-Nov-1900
30-Jul-1901
01-Dec-1901
19-Jan-1902
10-Nov-1902
Reported 22-Dec-1902
July-1903 
1904
04-Feb-1904
26-Mar-1905
06-Apr-1905
29-Jul-1905
Aug-1905
20-Jan-1906
28-Jan-1906
14-Apr-1906
1907
03-Feb-1907
05-Feb-1907
31-Dec-1908
15-Jul-1909
Ca. 1911
09-May-1911
25-Oct-1911
Reported 13-Jan-1912
26-Jan-1912
19-Feb-1912
30-Aug-1912
Reported 30-Nov-1912
21-Nov-1913
10-Jun-1914
09-Sep-1914
01-Jan-1915
10-Nov-1915
03-Apr-1916
23-Jun-1916
30-Jun-1916
01-Jul-1916
12-Jul-1916
09-Nov-1916
08-Dec-1916
30-Dec-1916
19-Sep-1918
05-Jan-1919
09-Jan-1919
07-Dec-1919
Reported 30-Dec-1919
1920
15-Jan-1920
03-Feb-1920
08-Mar-1920
29-Jun-1920
28-Aug-1921
04-Oct-1921
27-Nov-1921
11-Dec-1921
04-Jan-1922
15-Jan-1922
04-Feb-1922
02-Mar-1922
13-Mar-1922
06-May-1922
24-May 1922
17-Jun-1922
29-Sep-1922
05-Dec-1922
27-Jan-1923
Feb-1923
09-Jan-1924
29-Jan-1924
13-Feb-1924
22-Apr-1924
25-Apr-1924
21-Nov-1924
24-No

In [51]:
# Buscamos el mes en la columna 'date' con una función:

def month1(x):
    mon = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dic']
    res = 'unknown'
    for i in mon: 
        if i in x.lower(): 
            res = i
        elif 'summer' in x.lower(): 
            res = 'summer'
        else: 
            pass

    return res

shark.month = [month1(x) for x in shark.date]

shark.month.value_counts()

jul        317
aug        283
sep        264
unknown    261
jun        235
oct        220
jan        211
apr        211
nov        189
may        188
mar        174
feb        164
summer       5
Name: month, dtype: int64

In [53]:
for i in shark.year.unique():
    print(i)

0.0
1900.0
1901.0
1902.0
1903.0
1904.0
1905.0
1906.0
1907.0
1908.0
1909.0
1911.0
1912.0
1913.0
1914.0
1915.0
1916.0
1918.0
1919.0
1920.0
1921.0
1922.0
1923.0
1924.0
1925.0
1926.0
1927.0
1928.0
1929.0
1930.0
1931.0
1932.0
1933.0
1934.0
1935.0
1936.0
1937.0
1938.0
1939.0
1940.0
1941.0
1942.0
1943.0
1944.0
1945.0
1946.0
1947.0
1948.0
1949.0
1950.0
1951.0
1952.0
1953.0
1954.0
1955.0
1956.0
1957.0
1958.0
1959.0
1960.0
1961.0
1971.0
1962.0
1963.0
1964.0
1965.0
1966.0
1967.0
1968.0
1969.0
1970.0
1972.0
1973.0
1974.0
1975.0
1976.0
1977.0
1978.0
1979.0
1980.0
1981.0
1982.0
1983.0
1984.0
1985.0
1986.0
1987.0
1988.0
1989.0
1990.0
1991.0
1992.0
1993.0
1994.0
1995.0
1996.0
1997.0
1998.0
1999.0
2000.0
2001.0
2002.0
2003.0
2004.0
2005.0
2006.0
2007.0
2008.0
2009.0
2010.0
2011.0
2012.0
2013.0
2014.0
2015.0
2016.0
2017.0
2018.0


In [27]:
# Filtramos por debajo de 1900:

#shark.drop(index = shark[shark.year<1900].index, inplace = True)
#shark.shape, shark_ori.shape

In [28]:
# En este punto, ya podríamos utilizar la tabla para obtener información estadística de cara al objetivo. 
# Sin embargo, para dejarla mejor terminaremos de limpiar las columnas y rellenaremos lo que podamos de los 
# registros incompletos. 

# Reseteamos el índice porque en principio ya que no vamos a borrar las filas, y rellenamos las columnas 
# 'case_number', 'case_number.1', 'case_number.2' y 'original_order' con este índice de acuerdo a lo razonado 
# al principio del proceso.

shark.reset_index (inplace = True, drop = True)
shark['case_number'] = shark.index
shark['case_number.1'] = shark.index
shark['case_number.2'] = shark.index
shark['original_order'] = shark.index

shark.head()

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,day,month
0,0,Early 1930s,0.0,Unprovoked,BELIZE,NaN,NaN,Standing,a servant,M,16,FATAL,True,NaN,12' tiger shark,Mitchell-Hedges,ND-0026-Belize.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,NaN,unknown
1,1,1941-1942,0.0,Unprovoked,IRAQ,Basrah,Shatt-el Arab River near a small boat stand,Swimming,male,M,13,"FATAL, left leg bitten with severe blood loss",True,Afternoon,Bull shark,B.W. Coad & L.A.J. Al-Hassan,ND-0038-Shatt-al-Arab.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1,1,1,NaN,unknown
2,2,Before 1957,0.0,Provoked,CUBA,Havana Province,Cojimar,"Shark fishing, knocked overboard",Sandrillio,M,50,"FATAL, hip bitten PROVOKED INCIDENT",True,NaN,NaN,"F. Poli, pp.75, 81-83",ND-0051-Sandrillio.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2,2,2,NaN,unknown
3,3,1960s,0.0,Unprovoked,IRAQ,Basrah,Shatt-al-Arab River near Abu al Khasib,Swimming in section of river used for washing ...,male,M,16,Right leg lacerated & surgically amputated,False,Afternoon,Bull shark,B.W. Coad & L.A.J. Al-Hassan,ND-0063-Shatt-al-Arab.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3,3,3,NaN,unknown
4,4,1960s,0.0,Unprovoked,IRAQ,Basrah,Shatt-al-Arab River,Swimming naked near a date palm where many dat...,male,M,6,Arm severed,False,Afternoon,Bull shark,B.W. Coad & L.A.J. Al-Hassan,ND-0064-Shatt-al-Arab.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,4,4,4,NaN,unknown
